In [107]:
def read(f):
    for l in open(f):
        yield eval(l)
        
x = open("opt_dataset/opt.json")
i = 0
training = open("opt_dataset/small_opt.json",'w')

for l in x:
    if (i>=0) and (i<5000):
        training.write(l)
        if(i==0):
            print l
    i += 1
training.close()

{'userName': 'Roxanne Liu', 'recievedDate': '11/08/2015', 'docID': 'ICEB-2015-0002-12058', 'comment': 'It is very enssential for America to keep the international students. They are epquiped with skills and knowledge. The development of America relies on talents. Competition should not be the reason to ask them to leave.', 'postedDate': '11/11/2015'}



In [4]:
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model

comments = []
usernames = []
count = 0
for i in read('opt_dataset/small_opt.json'):
    comments.append(i['comment'])
    usernames.append(i['userName'])
    

In [5]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
stemmer = PorterStemmer()

for i in comments:
    r = ''.join([c for c in i.lower() if not c in punctuation])
    for w in r.split():
        # w = stemmer.stem(w)
        wordCount[w] += 1


In [99]:
labels = []
from sets import Set
vocab = Set()
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    for w in r.split():
        vocab.add(w)
numberpos = 0
numberneg = 0
for l in open('opt_dataset/labels_1-700.txt'):
    if(eval(l) == 1):
        labels.append(1)
        numberpos += 1
    else:
        labels.append(0)
        numberneg += 1

In [53]:
# Implementing bernoulli Naive Bayes. Another popular version is multinomial naive bayes
pos_wordCount = defaultdict(int)
neg_wordCount = defaultdict(int)
for w in vocab:
    pos_wordCount[w] = 0
    neg_wordCount[w] = 0
# calculating the probabilites
for i in range(450):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    bag = r.split()
    for w in vocab:
        if w in bag:
            if(labels[i] == 1):
                pos_wordCount[w] += 1
            else:
                neg_wordCount[w] += 1
    

In [64]:
# normalizing, with some kind of smoothing

pos_wordprob = defaultdict(float)
neg_wordprob = defaultdict(float)

for w in vocab:
    pos_wordprob[w] = 1.0*(pos_wordCount[w] + 0.01)/(1 + numberpos)
    neg_wordprob[w] = 1.0*(neg_wordCount[w] + 0.01)/(1 + numberneg)
posprob = numberpos*1.0/(numberpos+numberneg)
negprob = numberneg*1.0/(numberpos+numberneg)




In [65]:
import math
def predpos(words):
    sum = 0.0
    for w in vocab:
        if(w in words):
            sum += math.log(pos_wordprob[w])
        else:
            sum += math.log(1-pos_wordprob[w])
    return math.log(posprob) + sum
            
def predneg(words):
    sum = 0.0
    for w in vocab:
        if(w in words):
            sum += math.log(neg_wordprob[w])
        else:
            sum += math.log(1-neg_wordprob[w])
    return math.log(negprob) + sum
            
predictions = []            
for i in range(450):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    words = r.split()
    if(predpos(words) > predneg(words)):
        predictions.append(1)
    else:
        predictions.append(0)

In [69]:
for i in range(450):
    if(predictions[i] != labels[i]):
        print i,comments[i], "predicted: ", predictions[i], "Actual:", labels[i],"\n"

SyntaxError: invalid syntax (<ipython-input-69-e8f66a691ae8>, line 3)

In [124]:
# Multinomial naive bayes!

pos_wordCount_mult = defaultdict(int)
neg_wordCount_mult = defaultdict(int)
for w in vocab:
    pos_wordCount_mult[w] = 0
    neg_wordCount_mult[w] = 0
numberpos_mult = 0
numberneg_mult = 0
# calculating the probabilites
for i in range(600):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    bag = r.split()
    for w in bag:
        if(labels[i] == 1):
            pos_wordCount_mult[w] += 1
            numberpos_mult += 1
        else:
                neg_wordCount_mult[w] += 1
                numberneg_mult += 1
                
pos_wordprob_mult = defaultdict(float)
neg_wordprob_mult = defaultdict(float)

for w in vocab:
    pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
    neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)


In [125]:
import math
def predpos_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(pos_wordprob_mult[w])
    return (math.log(posprob) + sum)
            
def predneg_mult(words):
    sum = 0.0
    for w in words:
        if(w in vocab):
            sum += math.log(neg_wordprob_mult[w])
    return (math.log(negprob) + sum)
            
predictions_mult = []
LLP = []
LLN = []
for i in range(len(comments)):
    r = ''.join([c for c in comments[i].lower() if not c in punctuation])
    words = r.split()
    x = predpos_mult(words)
    y = predneg_mult(words)
    LLP.append(x)
    LLN.append(y)
    if(x > y):
        predictions_mult.append(1)
    else:
        predictions_mult.append(0)

In [119]:
# Semi supervised learning, Some kind of expectation maximization...
num_iter = 0
while(num_iter < 4):
    pos_wordCount_mult = defaultdict(int)
    neg_wordCount_mult = defaultdict(int)
    for w in vocab:
        pos_wordCount_mult[w] = 0
        neg_wordCount_mult[w] = 0
    numberpos_mult = 0
    numberneg_mult = 0
    # calculating the probabilites
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        bag = r.split()
        for w in bag:
            if(predictions_mult[i] == 1):
                pos_wordCount_mult[w] += 1
                numberpos_mult += 1
            else:
                    neg_wordCount_mult[w] += 1
                    numberneg_mult += 1

    pos_wordprob_mult = defaultdict(float)
    neg_wordprob_mult = defaultdict(float)

    for w in vocab:
        pos_wordprob_mult[w] = 1.0*(pos_wordCount_mult[w] + 1)/(len(vocab) + numberpos_mult)
        neg_wordprob_mult[w] = 1.0*(neg_wordCount_mult[w] + 1)/(len(vocab) + numberneg_mult)

    predictions_mult = []
    LLP = []
    LLN = []
    for i in range(len(comments)):
        r = ''.join([c for c in comments[i].lower() if not c in punctuation])
        words = r.split()
        x = predpos_mult(words)
        y = predneg_mult(words)
        LLP.append(x)
        LLN.append(y)
        if(x > y):
            predictions_mult.append(1)
        else:
            predictions_mult.append(0)
    
    num_iter += 1
    print "iteration number:",num_iter

iteration number: 1
iteration number: 2
iteration number: 3
iteration number: 4
iteration number: 5
iteration number: 6
iteration number: 7
iteration number: 8
iteration number: 9
iteration number: 10


In [126]:
train_accuracy = 0.0
test_accuracy = 0.0
for i in range(700):
    if(predictions_mult[i] == labels[i]):
        if i<600:
            train_accuracy += 1
        else:
            test_accuracy += 1
    elif(i>=600  and i<700):
        print comments[i], "True label:",labels[i],"Predicted label:",predictions_mult[i]
print "Test accuracy:",test_accuracy/100,"Train accuracy:", train_accuracy/600 

POTUS...just why are you selling us out at every opportunity? TPP...UN...Corporate power...runaway immigration...Zionist Israel...just for starters...The New World Order is NOT fantasy....Just stop your bushwhacking Fascism! How about training Americans who need it? True label: 0 Predicted label: 1
I see absolutely no reason for expanding the opportunities F-1 nonimmigrant students who merely make it much harder for United States young people who have such a difficult time finding work. Please do not allow additional visas or additional time for F-1 individuals to work in the U.S. True label: 0 Predicted label: 1
Enough is Enough. What ever happened to WE THE PEOPLE? True label: 0 Predicted label: 1
This is too much and redirects efforts away form our Country's educational system. The First priority should be to make sure we can not fill the STEM jobs with applicants within the Fifty State and Graduates within our boarders. I think this bill does not do enough for our citizens First an